## Problem Statement
Developing a Medical Research Assistant Using Retrieval-Augmented Generation (RAG)

#### Objective:
To create a Retrieval-Augmented Generation (RAG) application that functions as a medical research assistant, capable of answering complex medical questions using information retrieved from medical encyclopedia.

In [1]:
#LangChain
!pip install langchain
!pip install --upgrade langchain_community

#Hugging face
!pip install huggingface_hub

#Open AI
!pip install openai

#Pdf loader
!pip install pypdf
!pip install requests

#Vector DB
!pip install faiss-cpu

#Chromadb
!pip install chromadb

#Cohere
!pip install cohere

#Gradio
!pip install gradio

#Dependencies
!pip install sentence_transformers
!pip install langchain.chains.questions_answering
!pip install cython
!pip install unstructured[local-inference]
!pip install langchain.indexes
!pip install unstructured


ERROR: Could not find a version that satisfies the requirement langchain.chains.questions_answering (from versions: none)
ERROR: No matching distribution found for langchain.chains.questions_answering
ERROR: Could not find a version that satisfies the requirement langchain.indexes (from versions: none)
ERROR: No matching distribution found for langchain.indexes


In [2]:
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
import numpy as np
from pypdf import PdfReader
from tqdm import tqdm
def word_wrap(string, n_chars=72):
    # Wrap a string at the next space after n_chars
    if len(string) < n_chars:
        return string
    else:
        return string[:n_chars].rsplit(' ', 1)[0] + '\n' + word_wrap(string[len(string[:n_chars].rsplit(' ', 1)[0])+1:], n_chars)


In [3]:
from pypdf import PdfReader

reader = PdfReader("Encyclopedia of Medicine.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

print(word_wrap(pdf_texts[0]))

The GALE
ENCYCLOPEDIA
of MEDICINE
SECOND EDITION


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print(word_wrap(character_split_texts[10]))
print(f"\nTotal chunks: {len(character_split_texts)}")

alternative therapies, the encyclopedia features specific
Alternative
treatmentsections for diseases and condi-
tions that may be helped by
complementary therapies.
INCLUSION CRITERIA
A preliminary list of
diseases, disorders, tests and treat-
ments was compiled from a wide
variety of sources,
including professional medical guides and textbooks
as
well as consumer guides and encyclopedias. The general
advisory
board, made up of public librarians, medical
librarians and consumer
health experts, evaluated the top-
ics and made suggestions for
inclusion. The list was sorted
by category and sent to GEM2 medical
advisors, certified
physicians with various medical specialities, for
review.
Final selection of topics to include was made by the med-
ical
advisors in conjunction with the Gale Group editor.
ABOUT THE
CONTRIBUTORS
The essays were compiled by experienced medical
writers,
including physicians, pharmacists, nurses, and
other health care
professionals. GEM2 medical advisors

Total chu

In [5]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(word_wrap(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

the gale encyclopedia of medicine 2 ( gem2 ) is a one - stop source for
medical information on nearly 1, 700 common medical disorders,
conditions, tests, and treat - ments, including high - profile diseases
such as aids, alzheimer ’ s disease, cancer, and heart attack. this
ency - clopedia avoids medical jargon and uses language that laypersons
can understand, while still providing thor - ough coverage of each
topic. the gale encyclopedia of medicine 2 fills a gap between basic
consumer health resources, such as single - volume family medical
guides, and highly technical professional materials. scope almost 1,
700 full - length articles are included in the gale encyclopedia of
medicine 2, including disorders / conditions, tests / procedures, and
treatments / therapies. many common drugs are also covered, with
generic drug names appearing first and brand names following in
parentheses, eg. acetaminophen ( tylenol ). throughout the gale
encyclopedia of medicine 2, many prominent

Total c

In [6]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
#print(embedding_function([token_split_texts[10]]))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("info_medical", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

983

In [8]:
#
query = "What is PCR ?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(word_wrap(document))
    print('\n')

ic acids in a blood sample. numerous copies of a gene are made by
separating the two strands of dna containing the gene segment, marking
its location, using dna poly - merase to make a copy, and then
continuously replicating the copies. it is questionable whether pcr
will replace western blotting as the method of confirming aids diag -
noses. although pcr can detect the low number of per - sons ( 1 % )
with hiv infections that have not yet generated an antibody response to
the virus, the overwhelming majority of infected persons will be
detected by elisa screening within one to three months of infection. in
addition, pcr testing is based on present knowledge of the genetic
sequences in hiv. since the virus is continual - ly generating new
variants, pcr testing could yield a false negative in patients with
these new variants. in 1999, the u. s. food and drug administration (
fda ) approved an hiv home testing kit. the kit contains multiple
components, including material for specimen col

# Medical Research Assistant UI

In [14]:
import cohere
import gradio as gr
import chromadb

# Initialize the Cohere client
co = cohere.Client('icafxWDnWeIpLWCWEGBme4H8QU4ttQpUie8VK4C8')  # This is your trial API key

# Assume chroma_collection is properly initialized
# chroma_collection = ...

# Define the RAG function
def rag(query, retrieved_documents, model="command"):
    information = "\n\n".join(retrieved_documents)
    print("Retrieved Documents: ", retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": ("You are a helpful expert medical research assistant. Your users are asking questions "
                        "about information contained in an encyclopedia of medicine. You will be shown the user's question, "
                        "and the relevant information from encyclopedia of medicine. Answer the user's question "
                        "using only this information.")
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]

    response = co.chat(
        model=model,
        documents=messages,
        message=query, # Corrected to 'messages' instead of 'message'
    )

    return response.text  # Adjust to extract the text from the response

# Define the chatbot function
def chatbot(query):
    try:
        # Query Chroma to get retrieved documents
        results = chroma_collection.query(query_texts=[query], n_results=5)
        retrieved_documents = results['documents'][0]

        # Debug: print retrieved documents
        #print("Retrieved Documents: ", retrieved_documents)

        # Call the RAG function
        response = rag(query, retrieved_documents)
        source_text = "\n\n".join(retrieved_documents)  # Combine the retrieved documents as source text
        return response, source_text
    except Exception as e:
        # Debug: print exception details
        print("Error: ", e)
        return str(e), ""

# Set up the Gradio interface
iface = gr.Interface(
    fn=chatbot,  # Pass the user query to the chatbot function
    inputs=gr.Textbox(lines=2, placeholder="Ask a medical question..."),
    outputs=[
        gr.Textbox(label="Response", lines=4),
        gr.Textbox(label="Source Text", lines=10)
    ],
    title="Medical Research Assistant",
    description="Ask any medical-related question, and I will provide answers based on the relevant information."
)

# Launch the Gradio interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://41fce4210171e7c63b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
